In [16]:
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from openai import OpenAI

In [17]:
from project_secrets import GPT_4o_API_KEY

In [18]:
GPT_4o_MODEL = "openai/gpt-4o-mini"
client_4o = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=GPT_4o_API_KEY)

In [2]:
ds = load_dataset("ruslanmv/ai-medical-chatbot")

In [3]:
ds["train"][0]["Description"]

'Q. What does abutment of the nerve root mean?'

In [4]:
qa_pairs = [(entry["Patient"], entry["Doctor"]) for entry in ds["train"]]

qa_pairs[:3]

[('Hi doctor,I am just wondering what is abutting and abutment of the nerve root means in a back issue. Please explain. What treatment is required for\xa0annular bulging and tear?',
  'Hi. I have gone through your query with diligence and would like you to know that I am here to help you. For further information consult a neurologist online -->'),
 ('Hi doctor, I am a 22-year-old female who was diagnosed with hypothyroidism (genetic) when I was 12. Over the past five years, I have become around 50 pounds overweight and all of my attempts to lose have seemed to fail so I have given up, but my weight has stayed the same. There is so much information put there about losing weight with hypothyroidism but it all seems to conflict. I am so unsure as to what type of exercise and diet I should follow as a result but I still would like to lose weight, but most importantly have my body feel better. What can I do? I am currently on Levothyroxine, Buspar, and Benedryl.',
  'Hi. You have really don

In [6]:
questions, answers = zip(*qa_pairs)

In [9]:
vectorizer = TfidfVectorizer()
question_vectors = vectorizer.fit_transform(questions)

In [10]:
def find_best_match(user_input):
    user_vector = vectorizer.transform([user_input])  # Vectorize user input
    similarities = cosine_similarity(user_vector, question_vectors)  # Compute similarities
    best_match_idx = np.argmax(similarities)  # Get index of highest similarity
    return answers[best_match_idx]  # Return corresponding doctor's response

In [13]:
user_input = "Hi doctor, I fell on the ground and twisted my ankle. It is swollen and painful. What should I do?"
response = find_best_match(user_input)
print("Chatbot:", response)

Chatbot: your symptoms shows there is no fracture but we can't conclude completely but looks like peroneal muscle group strain.. just ice is putter side of leg from below the knee till the feet. support the alkle with a bandage or tape..


In [22]:
def medical_chatbot(user_input):
    similar_qa = find_best_match(user_input)
    response = client_4o.chat.completions.create(
        model=GPT_4o_MODEL,
        messages=[
            {
                "role": "system", "content": (
                "You are a medical professional. Use the following similar Q&A example to inform your response, "
                "but adapt it appropriately for the new question."
                )
            },
            {
                "role": "user", "content": (
                    f"Similar Q&A example:\nPatient Question: {user_input}\n"
                    f"Doctor's Response: {similar_qa}\n\nNew patient question: {user_input}"
                    )
            }
        ],
        temperature=0.7
    )
    return response.choices[0].message.content

In [24]:
user_query = "Hi doctor, I fell on the ground and twisted my ankle. It is swollen and painful. What should I do?"
ai_response = medical_chatbot(user_query)
print("Chatbot:", ai_response)

Chatbot: Doctor's Response: It sounds like you may have a sprain or strain in your ankle. While I can't diagnose without an examination, the swelling and pain you're experiencing are common symptoms. I recommend that you follow the R.I.C.E. method: Rest your ankle, Ice it to reduce swelling, Compress it with a bandage, and Elevate it above heart level when possible. If the pain persists, or if you are unable to put weight on it, I would advise you to see a healthcare professional for a thorough evaluation to rule out any fractures.
